In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

Checking out the dataset that was in two parts and then merging them with the concat() method, since they have the same columns

In [2]:
big_df_pt1 = pd.read_csv('../data/raw/df_final_web_data_pt_1.txt')
big_df_pt2 = pd.read_csv('../data/raw/df_final_web_data_pt_2.txt')
big_df_pt1

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
343136,2443347,465784886_73090545671,136329900_10529659391_316129,confirm,2017-03-31 15:15:46
343137,2443347,465784886_73090545671,136329900_10529659391_316129,step_3,2017-03-31 15:14:53
343138,2443347,465784886_73090545671,136329900_10529659391_316129,step_2,2017-03-31 15:12:08
343139,2443347,465784886_73090545671,136329900_10529659391_316129,step_1,2017-03-31 15:11:37


In [3]:
big_df_pt2

,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,2017-06-06 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,2017-06-01 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,2017-06-01 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,2017-06-01 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,2017-06-01 11:57:58
...,...,...,...,...,...
412259,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
412260,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
412261,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
412262,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [4]:
merged_df = pd.concat([big_df_pt1, big_df_pt2], axis=0)
merged_df

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
412259,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
412260,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
412261,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
412262,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [5]:
# Basic inspecting of the dataframe
def inspect_dataframe(merged_df):
    """
    Function to perform basic inspection on a DataFrame: 
    shape, column names, data types, and missing values.
    
    """

    print('Check the shape (rows, columns):')
    print(merged_df.shape)

    print('\nColumn names:')
    print(merged_df.columns)


    print('\nData types:')
    print(merged_df.dtypes)


    print('\nMissing values:')
    print(merged_df.isnull().sum())

# clean column names
def clean_column_names(merged_df):
    """
    Function to clean the column names of a DataFrame:
    - Convert to lowercase
    - Replace spaces with underscores
    - Remove or replace special characters with underscores

    """

    def clean_name(name):
        name = name.lower()
        name = name.replace(" ", "_")
        name = re.sub(r'[^a-z0-9_]', '_', name)
        return name
    
    merged_df.columns = [clean_name(col) for col in merged_df.columns]    
    return merged_df

# check unique and empty values
def check_unique_and_empty(merged_df):
    """
    Function to print the unique and empty values for each column in a DataFrame.
    
    """
    result = []
    
    for column in merged_df.columns:
        unique_values = merged_df[column].nunique()
        empty_values = merged_df[column].isna().sum()
        
        empty_rows = merged_df[column][merged_df[column].isna()].index.tolist()
        
        result.append({
            'Column': column,
            'Unique Values': unique_values,
            'Empty Values': empty_values,
            'Empty Row Indices': empty_rows
        })
    
    merged_df = pd.DataFrame(result)
    
    merged_df.set_index('Column', inplace=True)
    
print(inspect_dataframe(merged_df)) 
print(clean_column_names(merged_df))
print(check_unique_and_empty(merged_df))
display(merged_df)

Check the shape (rows, columns):
(755405, 5)

Column names:
Index(['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time'], dtype='object')

Data types:
client_id        int64
visitor_id      object
visit_id        object
process_step    object
date_time       object
dtype: object

Missing values:
client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64
None
        client_id             visitor_id                      visit_id  \
0         9988021   580560515_7732621733  781255054_21935453173_531117   
1         9988021   580560515_7732621733  781255054_21935453173_531117   
2         9988021   580560515_7732621733  781255054_21935453173_531117   
3         9988021   580560515_7732621733  781255054_21935453173_531117   
4         9988021   580560515_7732621733  781255054_21935453173_531117   
...           ...                    ...                           ...   
412259    9668240   388766751_9038881013   922267647_3096648104

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
412259,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
412260,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
412261,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
412262,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [6]:
merged_df.dtypes

client_id        int64
visitor_id      object
visit_id        object
process_step    object
date_time       object
dtype: object

In [7]:
# Changing the date_time column values from objects to 
merged_df['date_time'] = pd.to_datetime(merged_df['date_time'])
merged_df.dtypes

client_id                int64
visitor_id              object
visit_id                object
process_step            object
date_time       datetime64[ns]
dtype: object

In [8]:
merged_df= merged_df.sort_values(by=["client_id", "visit_id", "date_time"])
merged_df

,client_id,visitor_id,visit_id,process_step,date_time
285515,169,201385055_71273495308,749567106_99161211863_557568,start,2017-04-12 20:19:36
285514,169,201385055_71273495308,749567106_99161211863_557568,step_1,2017-04-12 20:19:45
285513,169,201385055_71273495308,749567106_99161211863_557568,step_2,2017-04-12 20:20:31
285512,169,201385055_71273495308,749567106_99161211863_557568,step_3,2017-04-12 20:22:05
285511,169,201385055_71273495308,749567106_99161211863_557568,confirm,2017-04-12 20:23:09
...,...,...,...,...,...
305392,9999875,738878760_1556639849,931268933_219402947_599432,step_1,2017-06-01 22:40:08
305391,9999875,738878760_1556639849,931268933_219402947_599432,step_1,2017-06-01 22:41:28
305390,9999875,738878760_1556639849,931268933_219402947_599432,step_2,2017-06-01 22:41:47
305389,9999875,738878760_1556639849,931268933_219402947_599432,step_3,2017-06-01 22:44:58


In [9]:
# Step 1: Identify 'start' steps that are followed by a 'step_1'
# First, shift the process_step column to check the next step
merged_df['next_step'] = merged_df.groupby('visit_id')['process_step'].shift(-1)

# Step 2: Filter the DataFrame to retain 'start' only if the next step is 'step_1'
filtered_df = merged_df[
    (merged_df['process_step'] != 'start') | (merged_df['next_step'] == 'step_1')
]

# Step 3: Drop the temporary 'next_step' column
filtered_df = filtered_df.drop(columns=['next_step'])

# Step 4: Drop duplicates if needed
filtered_df = filtered_df.drop_duplicates()

# Display the resulting DataFrame
filtered_df

,client_id,visitor_id,visit_id,process_step,date_time
285515,169,201385055_71273495308,749567106_99161211863_557568,start,2017-04-12 20:19:36
285514,169,201385055_71273495308,749567106_99161211863_557568,step_1,2017-04-12 20:19:45
285513,169,201385055_71273495308,749567106_99161211863_557568,step_2,2017-04-12 20:20:31
285512,169,201385055_71273495308,749567106_99161211863_557568,step_3,2017-04-12 20:22:05
285511,169,201385055_71273495308,749567106_99161211863_557568,confirm,2017-04-12 20:23:09
...,...,...,...,...,...
305392,9999875,738878760_1556639849,931268933_219402947_599432,step_1,2017-06-01 22:40:08
305391,9999875,738878760_1556639849,931268933_219402947_599432,step_1,2017-06-01 22:41:28
305390,9999875,738878760_1556639849,931268933_219402947_599432,step_2,2017-06-01 22:41:47
305389,9999875,738878760_1556639849,931268933_219402947_599432,step_3,2017-06-01 22:44:58


### For our project, we need to know how much time a client spends on each step and if they ever reached the last step.

In [10]:
# Valid steps order
valid_steps_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Initialize a list to store the results
time_spent = []

# Function to map steps to their order
def step_order(step):
    return valid_steps_order.index(step)

# Iterate over each unique combination of client, visitor, and visit
for (client_id, visitor_id, visit_id), group in filtered_df.groupby(['client_id', 'visitor_id', 'visit_id']):
    prev_time = None
    prev_step = None
    last_valid_step_order = -1  # Start with an invalid step order to force a check for the first step
    
    for _, row in group.iterrows():
        current_time = row['date_time']
        current_step = row['process_step']
        
        # Initialize is_error to False
        is_error = False
        
        # Check if the current step is a valid step and follows the correct order (no skipping)
        current_step_order = step_order(current_step)
        
        if prev_time is not None:
            # Calculate the time difference between steps
            time_diff = current_time - prev_time
            
            if current_step_order < last_valid_step_order:
                # If the current step goes backwards, just include the row (no error)
                time_spent.append({
                    'client_id': client_id,
                    'visitor_id': visitor_id,
                    'visit_id': visit_id,
                    'from_step': prev_step,
                    'to_step': current_step,
                    'time_spent': time_diff,
                    'is_error': True  # No error for backward steps
                })
            elif current_step_order > last_valid_step_order + 1:
                # If the step order skips (e.g., step_1 → step_3), **skip this row**
                continue  # Skip this row, don't append it
            elif current_step_order == last_valid_step_order:
                # If the steps to and from are the same, **skip this row**
                continue
            else:
                # Valid transition, append to the list
                time_spent.append({
                    'client_id': client_id,
                    'visitor_id': visitor_id,
                    'visit_id': visit_id,
                    'from_step': prev_step,
                    'to_step': current_step,
                    'time_spent': time_diff,
                    'is_error': False  # Valid transition, no error
                })
        
        # Update the last valid step order and previous time/step for the next iteration
        last_valid_step_order = current_step_order
        prev_time = current_time
        prev_step = current_step
        
        if prev_step is None:
            # For the first step, no previous step
            time_spent.append({
                'client_id': client_id,
                'visitor_id': visitor_id,
                'visit_id': visit_id,
                'from_step': None,
                'to_step': current_step,
                'time_spent': None,
                'is_error': False  # No error for the first step
            })

# Create a DataFrame with the results
time_spent_df = pd.DataFrame(time_spent)

# Drop rows with None values in 'from_step' and 'to_step'
time_spent_df = time_spent_df.dropna(subset=['from_step', 'to_step'])

# Display the resulting DataFrame
time_spent_df

,client_id,visitor_id,visit_id,from_step,to_step,time_spent,is_error
0,169,201385055_71273495308,749567106_99161211863_557568,start,step_1,0 days 00:00:09,False
1,169,201385055_71273495308,749567106_99161211863_557568,step_1,step_2,0 days 00:00:46,False
2,169,201385055_71273495308,749567106_99161211863_557568,step_2,step_3,0 days 00:01:34,False
3,169,201385055_71273495308,749567106_99161211863_557568,step_3,confirm,0 days 00:01:04,False
4,546,475037402_89828530214,731811517_9330176838_94847,start,step_1,0 days 00:00:10,False
...,...,...,...,...,...,...,...
487974,9999839,948322592_28990736246,715530422_68620416793_515645,step_3,confirm,0 days 00:00:35,False
487975,9999875,738878760_1556639849,931268933_219402947_599432,start,step_1,0 days 00:00:07,False
487976,9999875,738878760_1556639849,931268933_219402947_599432,step_1,step_2,0 days 00:01:39,False
487977,9999875,738878760_1556639849,931268933_219402947_599432,step_2,step_3,0 days 00:03:11,False


In [11]:
errors_occured = time_spent_df['is_error'].value_counts()
errors_occured

is_error
False    442751
True      45228
Name: count, dtype: int64

In [12]:
# Filter transitions to include only those that follow the valid steps order
valid_transitions = []

# Iterate over the time_spent_df and filter valid transitions
for _, row in time_spent_df.iterrows():
    from_step = row['from_step']
    to_step = row['to_step']
    
    # Check if the transition follows the valid steps order (i.e., from a step to the next one in order)
    if from_step is not None and to_step is not None:
        if valid_steps_order.index(to_step) == valid_steps_order.index(from_step) + 1:
            valid_transitions.append(row)

# Create a DataFrame with only valid transitions
valid_transitions_df = pd.DataFrame(valid_transitions)

# Calculate the average time spent on each transition
avg_time_spent = valid_transitions_df.groupby(['from_step', 'to_step'])['time_spent'].mean().round(0).reset_index()

# Rename the columns for clarity
avg_time_spent = avg_time_spent.rename(columns={'time_spent': 'avg_time_spent'})

# Display the resulting DataFrame
avg_time_spent

,from_step,to_step,avg_time_spent
0,start,step_1,0 days 00:00:32.047011243
1,step_1,step_2,0 days 00:00:39.737518411
2,step_2,step_3,0 days 00:01:31.883358069
3,step_3,confirm,0 days 00:01:57.315790788


Checking how much time on average a client might take on an error, on one of their visits to the platform, can also bring us insights

In [14]:
# Filter rows where 'is_error' is True
errors_df = time_spent_df[time_spent_df['is_error'] == True]

# Calculate the average time spent on errors
avg_time_per_error = errors_df['time_spent'].mean()

# Display the results
print(f"Average time spent per error: {avg_time_per_error} seconds")

errors_df

Average time spent per error: 0 days 00:03:15.106637481 seconds


,client_id,visitor_id,visit_id,from_step,to_step,time_spent,is_error
18,722,919259913_64837298108,984487154_55831795985_521110,step_2,step_1,0 days 00:00:16,True
19,722,919259913_64837298108,984487154_55831795985_521110,step_1,start,0 days 00:05:04,True
34,832,645971208_10952400740,704999523_65579948030_431837,step_2,step_1,0 days 00:00:11,True
41,1028,42237450_62128060588,557292053_87239438319_391157,step_3,step_1,0 days 00:01:51,True
43,1028,42237450_62128060588,557292053_87239438319_391157,step_2,step_1,0 days 00:00:09,True
...,...,...,...,...,...,...,...
487904,9998460,897717263_96593574747,626252558_3871240559_33704,step_1,start,0 days 00:03:41,True
487929,9998980,524732498_29551944764,630818873_69328760078_288792,step_1,start,0 days 00:01:37,True
487963,9999768,389637884_26725643037,85676722_11636430786_122704,step_3,step_2,0 days 00:01:58,True
487964,9999768,389637884_26725643037,85676722_11636430786_122704,step_2,step_1,0 days 00:00:01,True


Some clients didn't ever complete the process. Therefore, we need to ensure they are identified and see how many clients reached which step.

In [ ]:
# Identify clients who didn't complete the whole process (didn't reach "confirm")
completed_clients = time_spent_df[time_spent_df['to_step'] == 'confirm']['visit_id'].unique()

# Identify clients who didn't reach "confirm"
incomplete_clients = time_spent_df[~time_spent_df['visit_id'].isin(completed_clients)]['visit_id'].unique()

# Show which clients are incomplete
print(f"Clients who didn't complete the process: {incomplete_clients}")

# Check the last step they reached
last_steps = time_spent_df.groupby('visit_id')['to_step'].last()

# Show the last step for each client (whether complete or incomplete)
print(f"Last step for each client: \n{last_steps}")

In [ ]:
# Get the total number of unique visits
total_visits = time_spent_df['visit_id'].nunique()

# Identify the unique steps, excluding 'start' because it didn't make sense to keep it
steps = ['step_1', 'step_2', 'step_3', 'confirm'] 

# Initialize a dictionary to store the percentage of clients reaching each step
step_percentages = {}

# Iterate through each step and calculate the percentage
for step in steps:
    # Find unique visits that reached the step
    visits_reached_step = time_spent_df[time_spent_df['to_step'] == step]['visit_id'].nunique()
    
    # Calculate the percentage of visits who reached this step
    step_percentages[step] = (visits_reached_step / total_visits) * 100

# Print the results
for step, percentage in step_percentages.items():
    print(f"Percentage of visits who reached {step}: {percentage:.2f}%")

In [ ]:
# Now, for the general clients
# Get the total number of unique clients
total_clients = time_spent_df['client_id'].nunique()

# Identify the unique steps, excluding 'start' because it didn't make sense to keep it
steps = ['step_1', 'step_2', 'step_3', 'confirm'] 

# Initialize a dictionary to store the percentage of clients reaching each step
step_percentages = {}

# Iterate through each step and calculate the percentage
for step in steps:
    # Find unique clients that reached the step
    clients_reached_step = time_spent_df[time_spent_df['to_step'] == step]['client_id'].nunique()
    
    # Calculate the percentage of clients who reached this step
    step_percentages[step] = (clients_reached_step / total_clients) * 100

# Print the results
for step, percentage in step_percentages.items():
    print(f"Percentage of clients who reached {step}: {percentage:.2f}%")

## Exporting the tables 

In [ ]:
# filtered_df.to_csv('../data/clean/web_data_merged_filtered.csv')

In [ ]:
# time_spent_df.to_csv('../data/clean/time_spent.csv')